In [40]:
import pandas as pd
import numpy as np

import deepchecks as dc
from sklearn import datasets, linear_model, metrics, model_selection, pipeline, preprocessing, compose

In [46]:
data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
data = data.drop(['Name', 'Ticket', 'SibSp', 'Cabin','Embarked'],axis=1).dropna()
data.columns = data.columns.str.lower()
data[:3]

,passengerid,survived,pclass,sex,age,parch,fare
0,1,0,3,male,22.0,0,7.2500
1,2,1,1,female,38.0,0,71.2833
2,3,1,3,female,26.0,0,7.9250


In [47]:
target = 'survived'
features = data.columns.drop(target).tolist()
categoricals = ['sex', 'pclass', 'parch']

In [48]:
train, test = model_selection.train_test_split(data, test_size=0.33, random_state=42)

In [50]:
categorical_transformer = preprocessing.OneHotEncoder(handle_unknown="ignore")

preprocessor = compose.ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categoricals),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
model = pipeline.Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", linear_model.LogisticRegression(n_jobs=-1, max_iter=10_000))
        ]
)

model.fit(train[features], train[target])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sex', 'pclass',
                                                   'parch'])])),
                ('classifier', LogisticRegression(max_iter=10000, n_jobs=-1))])

In [51]:
suite = dc.suites.full_suite()

In [52]:
ds_train = dc.Dataset(train, label=target, cat_features=categoricals)
ds_test =  dc.Dataset(test,  label=target, cat_features=categoricals)

In [57]:
run = suite.run(train_dataset=ds_train, test_dataset=ds_test, model=model)
run

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]

/home/cristianexer/miniconda3/envs/ds/lib/python3.9/site-packages/deepchecks/utils/features.py:174: UserWarning:

Could not find built-in feature importance on the model, using permutation feature importance calculation



In [56]:
run.save_as_html('deepchecks.html')